In [1]:
import torch, torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10

from bgan.cnnTrainer import CnnTrainer
from bgan.vatCnnTrainer import VatCnnTrainer
from bgan.networkparts import layer13
import bgan.augLayers as augLayers
from bgan.schedules import cosLr, sigmoidConsRamp
import torch.optim as optim
import torch.nn as nn
%matplotlib inline
%load_ext autoreload
%autoreload 2

## CNN Configuration

In [2]:
savedir = '/home/maf388/tb-experiments/vatTesting/'

total_epochs = 1000
opt_constr = lambda params, base_lr: optim.SGD(params, base_lr, .9, weight_decay=1e-4, nesterov=True)
lr_lambda = cosLr(total_epochs, 1)

config = {'base_lr':.1, 'amntLab':4000, 
          'lab_BS':50, 'ul_BS':50, 'num_workers':0,
          'lr_lambda':lr_lambda, 'opt_constr':opt_constr,
          #'advEps':32, 'entMin':False, 
          }

baseCNN = layer13(numClasses=10)

fullCNN = nn.Sequential(
    augLayers.RandomTranslate(4),
    augLayers.RandomHorizontalFlip(),
    #augLayers.GaussianNoise(0.15),
    baseCNN,
)


## Dataset

In [3]:
img_size = 32
transform_dev = transforms.Compose(
    [transforms.Resize(img_size),
     transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

transform_train = transform_dev
# transform_train = transforms.Compose(
#     [transforms.RandomCrop(32, padding=4),
#      transforms.RandomHorizontalFlip(),
#      transform_dev])

pathToDataset = '/scratch/datasets/cifar10/'
trainset = CIFAR10(pathToDataset, download=True, transform=transform_train)
devset = CIFAR10(pathToDataset, train=False, download=True, transform=transform_dev)
testset = None

datasets = (trainset, devset, testset)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
trainer = CnnTrainer(fullCNN,datasets,savedir,**config)

In [5]:
#trainer.train(1000)

In [6]:
trainer.getDevsetAccuracy()

0.09982085987261147

In [ ]:
trainer2 = VatCnnTrainer(fullCNN,datasets,savedir,**config)

In [ ]:
from bgan.utils import to_var_gpu
_,(X,_) = to_var_gpu(next(trainer2.train_iter))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torchvision.utils as vutils
def show(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
grid = vutils.make_grid(X.cpu().data,normalize=True,range=(-2.5,2.5))
show(grid)

In [ ]:
from bgan.vatCnnTrainer import kl_div_withlogits, _l2_normalize, norm
from torch.autograd import Variable

eps = 32
adv = trainer2.getAdvPert(trainer2.CNN, X)
x_adv = X+eps*adv
rp = Variable(_l2_normalize(torch.randn(X.size())).cuda(),requires_grad=False)
x_rp = X+eps*rp
grid = vutils.make_grid(x_adv.cpu().data,normalize=True,range=(-2.5,2.5))
show(grid)

In [ ]:
trainer2.CNN.train(False)

logits = trainer2.CNN(X)
logits_adv = trainer2.CNN(x_adv)
logits_rnd = trainer2.CNN(x_rp)
adv_kl = kl_div_withlogits(logits.detach(), logits_adv)
random_kl = kl_div_withlogits(logits.detach(), logits_rnd)
print("kls %.4E, %.4E"%(adv_kl,random_kl))